### 1. Import dependencies

In [1]:
#%pip install stable-baselines3[extra]
#%pip install pyglet==1.5.27
#%pip install torch torchvision torchaudio

In [20]:
import os
import time
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy #testing the ai


### 2. load Environment

In [3]:
environment_name = "CartPole-v0"
env = gym.make(environment_name)

In [4]:
env.reset()

array([ 0.02673337, -0.04961589,  0.02822757,  0.03619322], dtype=float32)

In [7]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        time.sleep(0.01)
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))
env.close()

Episode:1 Score:16.0
Episode:2 Score:9.0
Episode:3 Score:24.0
Episode:4 Score:27.0
Episode:5 Score:21.0


Understanding the environment

In [8]:
env.action_space

Discrete(2)

In [9]:
env.action_space.sample()

1

In [10]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [11]:
env.observation_space.sample()

array([ 1.5875038e+00,  2.2700460e+38, -1.5976331e-01,  4.0767172e+37],
      dtype=float32)

## 3. Training

In [18]:
log_path = os.path.join('Training', 'Logs')

In [21]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [26]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 619  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
---------------------------------------
| time/                   |           |
|    fps                  | 462       |
|    iterations           | 2         |
|    time_elapsed         | 8         |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.0068386 |
|    clip_fraction        | 0.081     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.687    |
|    explained_variance   | 0.000212  |
|    learning_rate        | 0.0003    |
|    loss                 | 7.83      |
|    n_updates            | 10        |
|    policy_gradient_loss | -0.0129   |
|    value_loss           | 54.1      |
---------------------------------------
---------------------------------------

In [27]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [28]:
model.save(PPO_path)

In [29]:
del model

In [30]:
model = PPO.load(PPO_path, env=env)

### 5. Evaluation

In [34]:
evaluate_policy(model, env, n_eval_episodes=1, render=True)

(200.0, 0.0)

In [35]:
env.close()

### 6.Evaluation

In [42]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))
env.close()

Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[97.]
Episode:4 Score:[200.]
Episode:5 Score:[108.]


### 7. Viewing Logs in Tensorboard

In [43]:
training_log_path = os.path.join(log_path, 'PPO_1')

In [46]:
#!tensorboard --logdir={training_log_path}

### 8.Adding a callback to the training Stage